In [2]:
from google.colab import files
uploaded = files.upload()  # 一次性全选你的 14 个 CSV
print("已上传：", list(uploaded.keys()))

Saving 异构GA_2.csv to 异构GA_2.csv
Saving 异构DE_2.csv to 异构DE_2.csv
Saving 异构DE_1.csv to 异构DE_1.csv
Saving 异构GA_1.csv to 异构GA_1.csv
Saving 4岛DE_4.csv to 4岛DE_4.csv
Saving 4岛DE_3.csv to 4岛DE_3.csv
Saving 4岛DE_2.csv to 4岛DE_2.csv
Saving 4岛DE_1.csv to 4岛DE_1.csv
Saving 4岛GA_4.csv to 4岛GA_4.csv
Saving 4岛GA_3.csv to 4岛GA_3.csv
Saving 4岛GA_2.csv to 4岛GA_2.csv
Saving 4岛GA_1.csv to 4岛GA_1.csv
Saving 非岛屿模型单DE.csv to 非岛屿模型单DE.csv
Saving 非岛屿模型单GA.csv to 非岛屿模型单GA.csv
已上传： ['异构GA_2.csv', '异构DE_2.csv', '异构DE_1.csv', '异构GA_1.csv', '4岛DE_4.csv', '4岛DE_3.csv', '4岛DE_2.csv', '4岛DE_1.csv', '4岛GA_4.csv', '4岛GA_3.csv', '4岛GA_2.csv', '4岛GA_1.csv', '非岛屿模型单DE.csv', '非岛屿模型单GA.csv']


In [3]:
# -*- coding: utf-8 -*-
"""
Reproduce figures for GA/DE single vs four-island experiments (Rastrigin-7D).

How to use in Colab:
1) Upload all CSVs to the working directory (same folder as this script), namely:
   - 非岛屿模型单GA.csv
   - 非岛屿模型单DE.csv
   - 4岛GA_1.csv, 4岛GA_2.csv, 4岛GA_3.csv, 4岛GA_4.csv
   - 4岛DE_1.csv, 4岛DE_2.csv, 4岛DE_3.csv, 4岛DE_4.csv
   - 异构GA_1.csv, 异构GA_2.csv, 异构DE_1.csv, 异构DE_2.csv
2) (optional) !pip install lifelines  # if you want the log-rank table
3) Run: !python repro_figures.py
"""

import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional libs
try:
    from scipy.stats import norm
except Exception as _:
    norm = None

try:
    from lifelines.statistics import logrank_test
except Exception:
    logrank_test = None

# ----------------- IO & utils -----------------

def ensure_dir(p):
    if not os.path.exists(p):
        os.makedirs(p)

FIG_DIR = "figs"
OUT_DIR = "out"
ensure_dir(FIG_DIR)
ensure_dir(OUT_DIR)

RUN_KEYS = {"run","run_id","trial","rep","seed","exp","exp_id","id","ga_run","de_run","实验","实验编号","运行","运行编号","序号"}
GEN_KEYS = {"generation","gen","iteration","iter","t","代","代数","迭代","迭代数"}
BEST_KEYS = {"best","fbest","best_so_far","bestsofar","fitness","value","objective","obj","最优","最优值","最佳","目标值","适应度"}

def _pick_col(df, candidates):
    cols = [c for c in df.columns if isinstance(c, str)]
    low = {c: c.strip().lower() for c in cols}
    # exact
    for c in cols:
        if low[c] in candidates: return c
    # substring
    for c in cols:
        if any(tok in low[c] for tok in candidates): return c
    raise ValueError(f"Cannot find required column among {df.columns.tolist()} using keys={candidates}")

def read_csv_flex(path):
    df = pd.read_csv(path)
    run_col = _pick_col(df, RUN_KEYS)
    gen_col = _pick_col(df, GEN_KEYS)
    best_col = _pick_col(df, BEST_KEYS)
    d = df[[run_col, gen_col, best_col]].copy()
    d.columns = ["run_id","generation","best"]
    d["run_id"] = pd.to_numeric(d["run_id"], errors="coerce").astype("Int64")
    d["generation"] = pd.to_numeric(d["generation"], errors="coerce").astype("Int64")
    d["best"] = pd.to_numeric(d["best"], errors="coerce")
    d = d.dropna(subset=["run_id","generation","best"]).astype({"run_id":int,"generation":int,"best":float})
    return d

def track_single(path, lam):
    d = read_csv_flex(path).sort_values(["run_id","generation"])
    d["fbest"] = d.groupby("run_id")["best"].cummin()
    d["lam"] = lam
    return d[["run_id","generation","fbest","lam"]]

def track_islands(paths, lam):
    frames=[]
    for p in paths:
        d = read_csv_flex(p).sort_values(["run_id","generation"])
        d["fbest_i"] = d.groupby("run_id")["best"].cummin()
        frames.append(d[["run_id","generation","fbest_i"]])
    raw = pd.concat(frames, ignore_index=True)
    sys = raw.groupby(["run_id","generation"])["fbest_i"].min().reset_index().rename(columns={"fbest_i":"fbest"})
    sys["fbest"] = sys.groupby("run_id")["fbest"].cummin()
    sys["lam"] = lam
    return sys

def fe_hit_per_run(track, theta, FE_cap):
    """Return per-run first-hit FE and success flag (right-censor at FE_cap)."""
    lam = int(track["lam"].iloc[0])
    out = []
    for rid, grp in track.groupby("run_id"):
        grp = grp.sort_values("generation")
        hit = grp.loc[grp["fbest"] <= theta]
        if len(hit) > 0:
            g = int(hit["generation"].iloc[0])
            fe = g*lam
            succ = 1 if fe <= FE_cap else 0
            fe = min(fe, FE_cap)
        else:
            succ = 0
            fe = FE_cap
        out.append({"run_id":rid,"fe_hit":fe,"success":succ})
    return pd.DataFrame(out).sort_values("run_id")

def step_ecdf(ax, fe_hits, FE_cap, label, lw=2.5):
    """Draw step ECDF from per-run first-hit FEs (successes only), bounded by [0,FE_cap]."""
    x = sorted(list(fe_hits.loc[fe_hits["success"]==1, "fe_hit"].values))
    n = len(fe_hits)
    if len(x) == 0:
        ax.step([0, FE_cap], [0, 0], where="post", linewidth=lw, label=label)
        return
    ys = np.arange(1, len(x)+1)/n
    xs = [0] + x + [FE_cap]
    ys = [0] + list(ys) + [ys[-1]]
    ax.step(xs, ys, where="post", linewidth=lw, label=label)

def wilson_ci(k, n, z=1.96):
    if n == 0:
        return (0.0, 0.0)
    p = k/n
    denom = 1.0 + z**2/n
    center = (p + z**2/(2*n))/denom
    half = z/denom * math.sqrt(p*(1-p)/n + z**2/(4*n**2))
    return (max(0.0, center-half), min(1.0, center+half))

def two_prop_z(k1,n1,k2,n2):
    if norm is None:
        return (np.nan, np.nan)
    p_pool = (k1+k2)/(n1+n2) if (n1+n2)>0 else 0.0
    se = math.sqrt(p_pool*(1-p_pool)*(1/n1 + 1/n2)) if n1>0 and n2>0 else 0.0
    if se == 0:
        return (np.nan, np.nan)
    z = (k1/n1 - k2/n2)/se
    p = 2*(1 - norm.cdf(abs(z)))
    return (z, p)

# ----------------- Figures -----------------

def plot_single_ecdf(ga_track, de_track, FE_cap, theta, out_png):
    ga_hits = fe_hit_per_run(ga_track, theta, FE_cap)
    de_hits = fe_hit_per_run(de_track, theta, FE_cap)
    fig, ax = plt.subplots(figsize=(8.5,4.8))
    step_ecdf(ax, ga_hits, FE_cap, label=f"GA_single (SR@50k={ga_hits['success'].mean():.2f})", lw=2.8)
    step_ecdf(ax, de_hits, FE_cap, label=f"DE_single (SR@50k={de_hits['success'].mean():.2f})", lw=2.8)
    ax.set_xlim(0, FE_cap)
    ax.set_ylim(0, 1.0)
    ax.set_xlabel("Function evaluations (FEs)")
    ax.set_ylabel("Success rate (ECDF)")
    ax.set_title(f"Single-island ECDF (step) vs FEs at θ = {theta:.0e}")
    ax.axvline(FE_cap, linestyle="--", linewidth=1.5, alpha=0.7)
    ax.grid(True, axis="both", linestyle="--", alpha=0.35)
    ax.legend(frameon=True)
    fig.tight_layout()
    fig.savefig(out_png, dpi=600, bbox_inches="tight")
    plt.close(fig)
    return ga_hits, de_hits

def plot_islands_ecdf(de4_track, het4_track, ga4_track, FE_cap, theta, out_png):
    de4_hits = fe_hit_per_run(de4_track, theta, FE_cap)
    het4_hits = fe_hit_per_run(het4_track, theta, FE_cap)
    ga4_hits = fe_hit_per_run(ga4_track, theta, FE_cap)
    fig, ax = plt.subplots(figsize=(8.5,4.8))
    step_ecdf(ax, de4_hits, FE_cap, label="DE4_sync", lw=2.6)
    step_ecdf(ax, het4_hits, FE_cap, label="HET4_sync", lw=2.6)
    step_ecdf(ax, ga4_hits, FE_cap, label="GA4_sync", lw=2.6)
    ax.set_xlim(0, FE_cap)
    ax.set_ylim(0, 1.0)
    ax.set_xlabel("Function evaluations (FE)")
    ax.set_ylabel("Success rate (ECDF)")
    ax.set_title(f"Islands — Step ECDF vs FE (θ={theta:.0e}), full 5000 gens ({FE_cap:,} FEs)")
    ax.grid(True, axis="both", linestyle="--", alpha=0.35)
    ax.legend(frameon=True)
    fig.tight_layout()
    fig.savefig(out_png, dpi=600, bbox_inches="tight")
    plt.close(fig)
    return de4_hits, het4_hits, ga4_hits

def plot_de_single_spaghetti(de_track, out_png):
    fig, ax = plt.subplots(figsize=(8.6,4.8))
    # spaghetti
    for rid, grp in de_track.groupby("run_id"):
        ax.plot(grp["generation"], np.log10(np.maximum(grp["fbest"].values, 1e-12)),
                linewidth=1.0, alpha=0.35)
    # median across runs
    gens = sorted(de_track["generation"].unique())
    med = []
    for g in gens:
        med.append(np.median(np.log10(np.maximum(de_track.loc[de_track["generation"]==g, "fbest"].values, 1e-12))))
    ax.plot(gens, med, linewidth=3.0, alpha=0.9, label="Median", color="#b255a0")
    # theta lines
    for y, lab in [(-3, r"$\theta$=1e-03"), (-5, r"$\theta$=1e-05"), (-7, r"$\theta$=1e-07")]:
        ax.axhline(y, linestyle="--", linewidth=1.5, alpha=0.6, color="#d69200")
        ax.text(gens[-1]+30, y, lab, va="center")
    ax.set_xlabel("Generation")
    ax.set_ylabel(r"log$_{10}$ (best-so-far)")
    ax.set_title("Single DE — best-so-far over generations (30 runs)")
    ax.grid(True, axis="both", linestyle="--", alpha=0.3)
    ax.legend()
    fig.tight_layout()
    fig.savefig(out_png, dpi=600, bbox_inches="tight")
    plt.close(fig)

def boxplot_islands_FEhit(de4_hits, het4_hits, ga4_hits, theta, out_png, out_pdf=None):
    def succ_only(h): return h.loc[h["success"]==1, "fe_hit"].values
    data = [succ_only(de4_hits), succ_only(het4_hits), succ_only(ga4_hits)]
    labels = ["DE4_sync","HET4_sync","GA4_sync"]
    fig, ax = plt.subplots(figsize=(8.6,4.8))
    ax.boxplot(data, labels=labels, showfliers=False, widths=0.55)
    ax.set_ylabel(r"FE to first hit at $\theta={}$".format(f"{theta:.0e}"))
    ax.set_title("First-hit distributions (successes only); 30 runs per group")
    ax.grid(axis="y", linestyle="--", alpha=0.35)
    fig.tight_layout()
    fig.savefig(out_png, dpi=600, bbox_inches="tight")
    if out_pdf:
        fig.savefig(out_pdf, dpi=600, bbox_inches="tight")
    plt.close(fig)

# --------- HET events attribution (micro) ---------

def het_events_ga_vs_de(ga_paths, de_paths):
    frames = []
    for p in ga_paths:
        d = read_csv_flex(p).sort_values(["run_id","generation"])
        d["fbest_i"] = d.groupby("run_id")["best"].cummin()
        d["type"] = "GA_like"
        frames.append(d[["run_id","generation","fbest_i","type"]])
    for p in de_paths:
        d = read_csv_flex(p).sort_values(["run_id","generation"])
        d["fbest_i"] = d.groupby("run_id")["best"].cummin()
        d["type"] = "DE_like"
        frames.append(d[["run_id","generation","fbest_i","type"]])
    all_i = pd.concat(frames, ignore_index=True)

    # System best by (run, generation)
    sys = all_i.groupby(["run_id","generation"])["fbest_i"].min().reset_index().rename(columns={"fbest_i":"sysbest"})
    sys["prev"] = sys.groupby("run_id")["sysbest"].shift(1)
    sys["delta"] = sys["prev"] - sys["sysbest"]
    events = sys[(sys["delta"].notna()) & (sys["delta"] > 0)].copy()  # strict improvement

    # Attribute credit: islands whose fbest_i equals sysbest at that generation
    merged = events.merge(all_i, on=["run_id","generation"], how="left")
    merged = merged.loc[np.isclose(merged["fbest_i"], merged["sysbest"], rtol=0, atol=1e-12)]
    cnt = merged.groupby(["run_id","generation"]).size().reset_index(name="contributors")
    merged = merged.merge(cnt, on=["run_id","generation"], how="left")
    merged["credit"] = 1.0/merged["contributors"]
    # Aggregate by type
    agg_count = merged.groupby("type")["credit"].sum().reset_index(name="events_credit_sum")
    agg_meandelta = merged.groupby("type")["delta"].mean().reset_index(name="mean_delta")
    # Scatter
    scatter_df = merged[["run_id","generation","delta","type"]].copy()
    return agg_count, agg_meandelta, scatter_df

def plot_het_events_bars(agg_count, agg_meandelta, out_count_png, out_mean_png):
    fig, ax = plt.subplots(figsize=(8.4,4.6))
    ax.bar(agg_count["type"], agg_count["events_credit_sum"])
    ax.set_ylabel("# of improvement events (credit-summed)")
    ax.set_title("HET: Number of global improvements by group")
    ax.grid(axis="y", linestyle="--", alpha=0.35)
    fig.tight_layout()
    fig.savefig(out_count_png, dpi=600, bbox_inches="tight")
    plt.close(fig)

    fig, ax = plt.subplots(figsize=(8.4,4.6))
    ax.bar(agg_meandelta["type"], agg_meandelta["mean_delta"])
    ax.set_ylabel("Mean improvement per event (Δ/event)")
    ax.set_title("HET: Mean improvement magnitude by group")
    ax.grid(axis="y", linestyle="--", alpha=0.35)
    fig.tight_layout()
    fig.savefig(out_mean_png, dpi=600, bbox_inches="tight")
    plt.close(fig)

def plot_het_scatter(scatter_df, out_png):
    fig, ax = plt.subplots(figsize=(8.6,4.8))
    colors = {"DE_like":"#1f77b4", "GA_like":"#ff7f0e"}
    for tp, grp in scatter_df.groupby("type"):
        ax.scatter(grp["generation"], np.log10(np.maximum(grp["delta"].values, 1e-12)),
                   s=9, alpha=0.55, label=tp, marker="x" if tp=="DE_like" else "o",
                   color=colors.get(tp, None))
    ax.set_xlabel("Generation")
    ax.set_ylabel(r"log$_{10}(\Delta$ improvement)")
    ax.set_title("HET: Improvement magnitude over time")
    ax.grid(True, linestyle="--", alpha=0.3)
    ax.legend()
    fig.tight_layout()
    fig.savefig(out_png, dpi=600, bbox_inches="tight")
    plt.close(fig)

# --------- Anytime summary (log10-gap) ---------

def loggap_at_budget(track, FE_budget):
    lam = int(track["lam"].iloc[0])
    g_star = int(np.floor(FE_budget/lam))
    xs = []
    for rid, grp in track.groupby("run_id"):
        grp = grp.sort_values("generation")
        b = grp.loc[grp["generation"]<=g_star, "fbest"]
        val = float(b.iloc[-1]) if len(b)>0 else float(grp["fbest"].iloc[0])
        xs.append({"run_id":rid, "log10_gap": np.log10(max(val, 1e-12))})
    return pd.DataFrame(xs).sort_values("run_id")

def boxplot_anytime(data_dict, title, ylabel, out_png):
    labels = list(data_dict.keys())
    series = [np.asarray(data_dict[k]["log10_gap"]) for k in labels]
    fig, ax = plt.subplots(figsize=(8.4,4.6))
    ax.boxplot(series, labels=labels, showfliers=False, widths=0.55)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.grid(axis="y", linestyle="--", alpha=0.35)
    fig.tight_layout()
    fig.savefig(out_png, dpi=600, bbox_inches="tight")
    plt.close(fig)

# --------- Log-rank on time-to-hit (optional) ---------

def pairwise_logrank(fe_hits_dict, FE_cap, out_tex=None):
    """fe_hits_dict: {label: DataFrame[fe_hit, success]}"""
    if logrank_test is None:
        print("lifelines not installed; skip log-rank.")
        return None
    rows = []
    keys = list(fe_hits_dict.keys())
    for i in range(len(keys)):
        for j in range(i+1, len(keys)):
            a, b = keys[i], keys[j]
            da, db = fe_hits_dict[a], fe_hits_dict[b]
            T_a = da["fe_hit"].values
            E_a = da["success"].values.astype(int)
            T_b = db["fe_hit"].values
            E_b = db["success"].values.astype(int)
            res = logrank_test(T_a, T_b, event_observed_A=E_a, event_observed_B=E_b)
            rows.append({"comparison": f"{a} vs {b}", "p": res.p_value})
    df = pd.DataFrame(rows)
    if out_tex:
        with open(out_tex, "w", encoding="utf-8") as f:
            f.write("\\begin{table}[t]\n\\centering\n")
            f.write("\\caption{Log-rank tests on time-to-hit (right-censored at %dk FEs).}\\n" % (FE_cap//1000))
            f.write("\\label{tab:logrank}\n\\footnotesize\n\\begin{tabular}{lc}\n\\hline\n")
            f.write("Comparison & $p$ \\\\\n\\hline\n")
            for _,r in df.iterrows():
                pstr = f"{r['p']:.2e}" if r['p']>=1e-15 else "$<1\\times10^{-15}$"
                f.write(f"{r['comparison']} & {pstr} \\\\\n")
            f.write("\\hline\n\\end{tabular}\n\\end{table}\n")
    return df

# ----------------- Main pipeline -----------------

def main():
    # File mapping (relative paths)
    ga_single_path = "非岛屿模型单GA.csv"
    de_single_path = "非岛屿模型单DE.csv"

    ga4_paths = ["4岛GA_1.csv","4岛GA_2.csv","4岛GA_3.csv","4岛GA_4.csv"]
    de4_paths = ["4岛DE_1.csv","4岛DE_2.csv","4岛DE_3.csv","4岛DE_4.csv"]
    het_ga_paths = ["异构GA_1.csv","异构GA_2.csv"]
    het_de_paths = ["异构DE_1.csv","异构DE_2.csv"]

    # Load tracks
    ga_single = track_single(ga_single_path, lam=10)
    de_single = track_single(de_single_path, lam=10)

    ga4 = track_islands(ga4_paths, lam=40)
    de4 = track_islands(de4_paths, lam=40)
    het4 = track_islands(het_ga_paths + het_de_paths, lam=40)  # system track for ECDF

    # ---- Single ECDFs (0..50k) ----
    FE_cap_single = 50_000
    for theta in [1e-3, 1e-5, 1e-7]:
        out_png = os.path.join(FIG_DIR, f"ECDF_single_FE_theta_{theta:.0e}_50k.png").replace("+0", "")
        plot_single_ecdf(ga_single, de_single, FE_cap_single, theta, out_png)

    # ---- Four-island ECDFs (0..200k) ----
    FE_cap_islands = 200_000
    for theta in [1e-3, 1e-5, 1e-7]:
        out_png = os.path.join(FIG_DIR, f"ISLANDS_ECDF_STEP_FE_theta_{theta:.0e}_FULL200k.png").replace("+0", "")
        plot_islands_ecdf(de4, het4, ga4, FE_cap_islands, theta, out_png)

    # ---- Single DE spaghetti ----
    plot_de_single_spaghetti(de_single, os.path.join(FIG_DIR, "DE_single_spaghetti_log10.png"))

    # ---- Islands FE_hit box @ theta=1e-5 ----
    de4_hits, het4_hits, ga4_hits = plot_islands_ecdf(
        de4, het4, ga4, FE_cap_islands, 1e-5,
        os.path.join(FIG_DIR, "ISLANDS_ECDF_for_box_theta_1e-5.png")
    )
    boxplot_islands_FEhit(de4_hits, het4_hits, ga4_hits, theta=1e-5,
        out_png=os.path.join(FIG_DIR, "BOX_islands_FEhit_theta1e-5.png"),
        out_pdf=os.path.join(FIG_DIR, "BOX_islands_FEhit_theta1e-5.pdf"))

    # ---- HET micro: events, mean delta, scatter ----
    agg_count, agg_meandelta, scatter_df = het_events_ga_vs_de(het_ga_paths, het_de_paths)
    agg_count.to_csv(os.path.join(OUT_DIR, "HET_events_count.csv"), index=False)
    agg_meandelta.to_csv(os.path.join(OUT_DIR, "HET_mean_delta.csv"), index=False)
    scatter_df.to_csv(os.path.join(OUT_DIR, "HET_events_scatter.csv"), index=False)
    plot_het_events_bars(agg_count, agg_meandelta,
        out_count_png=os.path.join(FIG_DIR, "events_count.png"),
        out_mean_png=os.path.join(FIG_DIR, "mean_delta.png"))
    plot_het_scatter(scatter_df, os.path.join(FIG_DIR, "scatter_logdelta_vs_gen.png"))

    # ---- Anytime summary ----
    # Single @50k
    gaps_ga1 = loggap_at_budget(ga_single, 50_000)
    gaps_de1 = loggap_at_budget(de_single, 50_000)
    pd.concat({"GA_single":gaps_ga1.describe(), "DE_single":gaps_de1.describe()}, axis=1).to_csv(
        os.path.join(OUT_DIR, "ANYTIME_single_loggap_50k_summary.csv"))
    boxplot_anytime({"GA_single":gaps_ga1,"DE_single":gaps_de1},
        title=r"Anytime log$_{10}$-gap at 50k FEs (single-island)",
        ylabel=r"log$_{10}$(best-so-far)",
        out_png=os.path.join(FIG_DIR, "ANYTIME_single_loggap_50k.png"))
    # Islands @30k (pre-saturation)
    gaps_de4 = loggap_at_budget(de4, 30_000)
    gaps_het4 = loggap_at_budget(het4, 30_000)
    gaps_ga4 = loggap_at_budget(ga4, 30_000)
    pd.concat({"DE4_sync":gaps_de4.describe(), "HET4_sync":gaps_het4.describe(), "GA4_sync":gaps_ga4.describe()}, axis=1).to_csv(
        os.path.join(OUT_DIR, "ANYTIME_islands_loggap_30k_summary.csv"))
    boxplot_anytime({"DE4_sync":gaps_de4,"HET4_sync":gaps_het4,"GA4_sync":gaps_ga4},
        title=r"Anytime log$_{10}$-gap at 30k FEs (four-island)",
        ylabel=r"log$_{10}$(system best-so-far)",
        out_png=os.path.join(FIG_DIR, "ANYTIME_islands_loggap_30k.png"))

    # ---- SR@50k tables (compact Wilson CI) ----
    def sr_ci(track, theta, FE_cap):
        hits = fe_hit_per_run(track, theta, FE_cap)
        k = int(hits["success"].sum()); n = len(hits)
        ci = wilson_ci(k,n)
        return k, n, k/n, ci
    # Single table
    rows = []
    for theta in [1e-3,1e-5,1e-7]:
        k1,n1,sr1,ci1 = sr_ci(ga_single, theta, FE_cap_single)
        k2,n2,sr2,ci2 = sr_ci(de_single, theta, FE_cap_single)
        rows.append({"theta":f"{theta:.0e}", "GA_single":f"{sr1:.2f} [{ci1[0]:.2f}, {ci1[1]:.2f}]",
                     "DE_single":f"{sr2:.2f} [{ci2[0]:.2f}, {ci2[1]:.2f}]"})
    pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR,"single_SR50k_compact.csv"), index=False)
    # Islands table
    rows = []
    for theta in [1e-3,1e-5,1e-7]:
        de_sr = sr_ci(de4, theta, FE_cap_islands)
        het_sr = sr_ci(het4, theta, FE_cap_islands)
        ga_sr = sr_ci(ga4, theta, FE_cap_islands)
        rows.append({"theta":f"{theta:.0e}",
                     "DE4_sync":f"{de_sr[2]:.2f} [{de_sr[3][0]:.2f}, {de_sr[3][1]:.2f}]",
                     "HET4_sync":f"{het_sr[2]:.2f} [{het_sr[3][0]:.2f}, {het_sr[3][1]:.2f}]",
                     "GA4_sync":f"{ga_sr[2]:.2f} [{ga_sr[3][0]:.2f}, {ga_sr[3][1]:.2f}]"})
    pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR,"islands_SR50k_compact.csv"), index=False)

    # ---- (optional) Pairwise log-rank at theta=1e-5 (200k censor) ----
    if logrank_test is not None:
        de_hits = fe_hit_per_run(de4, 1e-5, FE_cap_islands)
        het_hits = fe_hit_per_run(het4, 1e-5, FE_cap_islands)
        ga_hits = fe_hit_per_run(ga4, 1e-5, FE_cap_islands)
        pairwise_logrank({"DE4_sync":de_hits,"HET4_sync":het_hits,"GA4_sync":ga_hits},
                         FE_cap_islands, out_tex=os.path.join(OUT_DIR,"logrank_islands_theta1e5.tex"))

    print("Done. Figures -> ./figs , tables -> ./out")

if __name__ == "__main__":
    main()


/tmp/ipython-input-2220706836.py:211: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data, labels=labels, showfliers=False, widths=0.55)
/tmp/ipython-input-2220706836.py:308: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(series, labels=labels, showfliers=False, widths=0.55)
/tmp/ipython-input-2220706836.py:308: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(series, labels=labels, showfliers=False, widths=0.55)


Done. Figures -> ./figs , tables -> ./out
